In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License")
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https:  ##www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http:  ##colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https:  ##www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https:  ##github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https:  ##www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

## Script for downloading CHIRPS data in TX

- This script requires a file "TX_grids.csv" listing CPC grid codes in TX.
- It also requires a Google Earth Engine asset "users/ramaraja/grids" of CPC grids that can be created from "official_RMA_RI_grid.shp"
- Files are saved using file names like "CHIRPS_precip_TX_2012_625.csv" and must then be added to the `prf-ri` repo, `data` folder.

In [1]:
!pip install geojson

In [2]:
import os
import ee
import geemap
import pandas as pd
import geopandas as gpd
import shapely.geometry as shp
import geojson
import itertools

file paths and constants

In [3]:
gee_project = 'zari-331802' ## GEE project
grid_path = '/content/gdrive/MyDrive/PRF-RI/TX-grids.csv' ## state CPC grids

state_prefix = 'TX'

state_grid_chirps_csv = '/content/gdrive/MyDrive/PRF-RI/' + state_prefix + '_CHIRPS_df.csv' ## output file with CHIRPS lat/long coordinates

chirps_tracker = '/content/gdrive/MyDrive/PRF-RI/' + state_prefix + '_CHIRPS_tracker.csv'

chirps_start_year = 1981
chirps_end_year = 2021
chirps_all_years = list(range(chirps_start_year, chirps_end_year + 1))

## 2-month intervals used in PRF-RI
dateMap={
  625:{
   "startDate": '01-01',
   "endDate": '02-28',
   "caption": "Sum January & February Precipitation"
  },
  626:{
   "startDate": '02-01',
   "endDate": '03-31',
   "caption": "Sum February & March Precipitation"
  },
  627:{
   "startDate": '03-01',
   "endDate": '04-30',
   "caption": "Sum March & April Precipitation"
  },
  628:{
   "startDate": '04-01',
   "endDate": '05-31',
   "caption": "Sum April & May Precipitation"
  },
  629:{
   "startDate": '05-01',
   "endDate": '06-30',
   "caption": "Sum May & June Precipitation"
  },
  630:{
   "startDate": '06-01',
   "endDate": '07-31',
   "caption": "Sum June & July Precipitation"
  },
  631:{
   "startDate": '07-01',
   "endDate": '08-31',
   "caption": "Sum July & August Precipitation"
  },
  632:{
   "startDate": '08-01',
   "endDate": '09-30',
   "caption": "Sum August & September Precipitation"
  },
  633:{
   "startDate": '09-01',
   "endDate": '10-31',
   "caption": "Sum September & October Precipitation"
  },
  634:{
   "startDate": '10-01',
   "endDate": '11-30',
   "caption": "Sum October & November Precipitation"
  },
  635:{
   "startDate": '11-01',
   "endDate": '12-31',
   "caption": "Sum November & December Precipitation"
  }
};

Mount Google Drive and authenticate GEE

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
ee.Authenticate()  # Only needed if you haven't authenticated in this session
ee.Initialize(project = gee_project)

In [6]:
Map = geemap.Map()

Load in state CPC grid IDs

In [7]:
## load grid codes
state_grid_codes = pd.read_csv(grid_path)
state_grid_codes_list = state_grid_codes.GRIDCODE.tolist()
print(len(state_grid_codes_list))
print(state_grid_codes_list)



1163
[7030, 7031, 7032, 8223, 8224, 8225, 8226, 8227, 8228, 8229, 8230, 8231, 9421, 9422, 9423, 9424, 9425, 9426, 9427, 9428, 9429, 9430, 9431, 9432, 9433, 10607, 10608, 10618, 10619, 10620, 10621, 10622, 10623, 10624, 10625, 10626, 10627, 10628, 10629, 10630, 10631, 10632, 10633, 10634, 10635, 10636, 10637, 10638, 10639, 10640, 11802, 11803, 11804, 11805, 11806, 11807, 11808, 11809, 11810, 11811, 11812, 11813, 11814, 11815, 11816, 11817, 11818, 11819, 11820, 11821, 11822, 11823, 11824, 11825, 11826, 11827, 11828, 11829, 11830, 11831, 11832, 11833, 11834, 11835, 11836, 11837, 11838, 11839, 11840, 11841, 11842, 11843, 11844, 11845, 12998, 12999, 13000, 13001, 13002, 13003, 13004, 13005, 13006, 13007, 13008, 13009, 13010, 13011, 13012, 13013, 13014, 13015, 13016, 13017, 13018, 13019, 13020, 13021, 13022, 13023, 13024, 13025, 13026, 13027, 13028, 13029, 13030, 13031, 13032, 13033, 13034, 13035, 13036, 13037, 13038, 13039, 13040, 13041, 13042, 13043, 13044, 13045, 13046, 14194, 14195, 1419

Filter GEE grids asset to grids in state

In [8]:
## read in grid IDS for state
grids = ee.FeatureCollection("users/ramaraja/grids");

## filter grids feature collection to state grids
grids_state = grids.filter(
    ee.Filter.inList('GRIDCODE', ee.List(state_grid_codes_list))
)

print(grids_state.size().getInfo())
print(grids_state.first().getInfo())

1163
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-97.75000090456851, 25.75000207159434], [-97.68750101106008, 25.750002043839366], [-97.62500117066254, 25.75000205101646], [-97.56250132512623, 25.750002022892797], [-97.50000141810179, 25.750002068355638], [-97.50000141810179, 26.000001539994695], [-97.56250132512623, 26.000001547708635], [-97.62500117066254, 26.000001593717293], [-97.68750101106008, 26.000001569111248], [-97.75000090456851, 26.00000154393282], [-97.75000090456851, 25.75000207159434]]]}, 'id': '0000000000000000594b', 'properties': {'GRIDCODE': 7030, 'X_MAX': -97.5, 'X_MIN': -97.75, 'Y_MAX': 26, 'Y_MIN': 25.75}}


Convert GEE state grids to a pandas df, dropping spatial info (not needed)

In [9]:
## convert state grids to pandas df, dropping spatial info (not needed)

def fc_to_df(fc):
  """Converts a GEE FeatureCollection to a Pandas DataFrame, dropping spatial information.

  Args:
    fc: An ee.FeatureCollection.

  Returns:
    A Pandas DataFrame containing the feature attributes.
  """

  # Get the feature collection as a list of dictionaries
  fc_list = fc.getInfo()['features']

  # Extract properties (attributes) from each feature
  data = [f['properties'] for f in fc_list]

  # Create a Pandas DataFrame
  df = pd.DataFrame(data)

  return df

state_grid_df = fc_to_df(grids_state)
print(state_grid_df.shape)
state_grid_df.head()



(1163, 5)


,GRIDCODE,X_MAX,X_MIN,Y_MAX,Y_MIN
0,7030,-97.50,-97.75,26.0,25.75
1,7031,-97.25,-97.50,26.0,25.75
2,7032,-97.00,-97.25,26.0,25.75
3,8223,-99.25,-99.50,27.0,26.75
4,8224,-99.00,-99.25,27.0,26.75


## create a CHIRPS level data frame that creates a 5 x 5 sub-grids within each CPC grid

In [10]:


def expand_dataframe(df):
  """Expands a DataFrame by creating all possible combinations of 'CHIRPS_OFFSET_X' and 'CHIRPS_OFFSET_Y' from 0 to 4.

  Args:
    df: The original DataFrame.

  Returns:
    An expanded DataFrame with new rows for all combinations of offsets.
  """

  # Create all possible combinations of offsets
  offsets = list(itertools.product(range(5), repeat=2))

  # Create a new DataFrame to hold the expanded data
  expanded_df = pd.DataFrame(columns=df.columns.tolist() + ['CHIRPS_OFFSET_X', 'CHIRPS_OFFSET_Y'])

  # Iterate over the original DataFrame and expand each row
  for offset_x, offset_y in offsets:
    df_copy = df.copy()
    df_copy['CHIRPS_OFFSET_X'] = offset_x
    df_copy['CHIRPS_OFFSET_Y'] = offset_y
    expanded_df = pd.concat([expanded_df, df_copy])

  return expanded_df

state_grid_chirps_df = expand_dataframe(state_grid_df)

## add columns for 'centroid' of CHIRPS pixel
## centroid may not be exact due to rounding but should give us the correct value for the CHIRPS pixel
state_grid_chirps_df['CHIRPS_LON'] = state_grid_chirps_df['X_MIN'] + (state_grid_chirps_df['CHIRPS_OFFSET_X'] * 0.05) + (0.05/2)
state_grid_chirps_df['CHIRPS_LAT'] = state_grid_chirps_df['Y_MIN'] + (state_grid_chirps_df['CHIRPS_OFFSET_Y'] * 0.05) + (0.05/2)

state_grid_chirps_df.to_csv(state_grid_chirps_csv)

print(state_grid_chirps_df.shape)
state_grid_chirps_df[state_grid_chirps_df['GRIDCODE'] == 7030].head(10)




(29075, 9)


,GRIDCODE,X_MAX,X_MIN,Y_MAX,Y_MIN,CHIRPS_OFFSET_X,CHIRPS_OFFSET_Y,CHIRPS_LON,CHIRPS_LAT
0,7030,-97.5,-97.75,26.0,25.75,0,0,-97.725,25.775
0,7030,-97.5,-97.75,26.0,25.75,0,1,-97.725,25.825
0,7030,-97.5,-97.75,26.0,25.75,0,2,-97.725,25.875
0,7030,-97.5,-97.75,26.0,25.75,0,3,-97.725,25.925
0,7030,-97.5,-97.75,26.0,25.75,0,4,-97.725,25.975
0,7030,-97.5,-97.75,26.0,25.75,1,0,-97.675,25.775
0,7030,-97.5,-97.75,26.0,25.75,1,1,-97.675,25.825
0,7030,-97.5,-97.75,26.0,25.75,1,2,-97.675,25.875
0,7030,-97.5,-97.75,26.0,25.75,1,3,-97.675,25.925
0,7030,-97.5,-97.75,26.0,25.75,1,4,-97.675,25.975


Visualize CHIRPS points for 3 grid cells (TX only)

In [12]:
## convert pandas df to ee feature collection
state_grid_chirps_fc = geemap.df_to_ee(state_grid_chirps_df[state_grid_chirps_df['GRIDCODE'] <= 7032], latitude='CHIRPS_LAT', longitude='CHIRPS_LON')

print(state_grid_chirps_fc.size().getInfo())

Map.addLayer(state_grid_chirps_fc)
Map.centerObject(state_grid_chirps_fc, 10)

Map

75


Map(center=[25.875159908185857, -97.375], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

create chunks of CPC grid ID's for looping

This prevents GEE memory issues.

In [13]:
## loop over sets of CPC grid ID's, intervals, and years

chunk_size = 100

def chunks(lst, n):
  """Yield successive n-sized chunks from lst."""
  for i in range(0, len(lst), n):
    yield lst[i:i + n]

grid_chunks = list(chunks(state_grid_codes_list, chunk_size))
for chunk in grid_chunks:
  print(chunk)#for i in range((len(state_grid_codes_list) // grid_chunk_size + 1):
  print(len(chunk))


[7030, 7031, 7032, 8223, 8224, 8225, 8226, 8227, 8228, 8229, 8230, 8231, 9421, 9422, 9423, 9424, 9425, 9426, 9427, 9428, 9429, 9430, 9431, 9432, 9433, 10607, 10608, 10618, 10619, 10620, 10621, 10622, 10623, 10624, 10625, 10626, 10627, 10628, 10629, 10630, 10631, 10632, 10633, 10634, 10635, 10636, 10637, 10638, 10639, 10640, 11802, 11803, 11804, 11805, 11806, 11807, 11808, 11809, 11810, 11811, 11812, 11813, 11814, 11815, 11816, 11817, 11818, 11819, 11820, 11821, 11822, 11823, 11824, 11825, 11826, 11827, 11828, 11829, 11830, 11831, 11832, 11833, 11834, 11835, 11836, 11837, 11838, 11839, 11840, 11841, 11842, 11843, 11844, 11845, 12998, 12999, 13000, 13001, 13002, 13003]
100
[13004, 13005, 13006, 13007, 13008, 13009, 13010, 13011, 13012, 13013, 13014, 13015, 13016, 13017, 13018, 13019, 13020, 13021, 13022, 13023, 13024, 13025, 13026, 13027, 13028, 13029, 13030, 13031, 13032, 13033, 13034, 13035, 13036, 13037, 13038, 13039, 13040, 13041, 13042, 13043, 13044, 13045, 13046, 14194, 14195, 1419

create tracker file if it doesn't exist.

tracker file shows which year - interval combinations have been completed.

In [14]:
if os.path.exists(chirps_tracker):
  print('file exists')
  tracker_df = pd.read_csv(chirps_tracker, index_col=False)
else:
  year_interval_combinations = list(itertools.product(chirps_all_years, list(dateMap.keys()) ))
  tracker_df = pd.DataFrame(year_interval_combinations, columns=['year', 'interval'])
  tracker_df['state'] = state_prefix
  tracker_df['completed'] = False
  tracker_df.to_csv(chirps_tracker, index = False)
 #   tracker_df = pd.DataFrame()
tracker_df.head(20)


file exists


,year,interval,state,completed
0,1981,625,TX,True
1,1981,626,TX,True
2,1981,627,TX,True
3,1981,628,TX,True
4,1981,629,TX,True
5,1981,630,TX,True
6,1981,631,TX,True
7,1981,632,TX,True
8,1981,633,TX,True
9,1981,634,TX,True


functions for CHIRPS data extraction

In [ ]:
## loop over sets of CPC grid ID's, intervals, and years


## interval and year loop

def add_image_value(feature):
  point = feature.geometry()
  value = chirps_sum.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=point,
      scale=25  # Adjust scale as needed
  )
  precip_value = value.get('precipitation')
  return feature.set('CHIRPS_precip', precip_value)
 # return feature.setMultiProperties(value)


def combine_dataframes(data_list):
  """Combines a list of DataFrames into a single DataFrame.

  Args:
    data_list: A list of DataFrames.

  Returns:
    A combined DataFrame.
  """

  combined_data = []
  for df in data_list:
    combined_data.extend(df.to_dict('records'))
  return pd.DataFrame(combined_data)


def extract_chirps_year_interval(year, interval, current_grid_chunks):
  """Extracts CHIRPS values for all grid cells for a given year and interval
  saves extracted values to csv

  Args:
    year: the year of CHIRPS data
    interval: the two-month interavl for CHIRPS data

  Returns:
    True if completed successfully
  """
  ## basically we create a separate df for extracted values in each 'chunk' of CPC grid values
  year_interval_dfs = []
  year_interval_csv_name = '/content/gdrive/MyDrive/PRF-RI/CHIRPS_precip_' + state_prefix + '_' + str(year) + '_' + str(interval) + '.csv'

  for chunk in current_grid_chunks:
    print(chunk)
    state_grid_chirps_df_chunk = state_grid_chirps_df[state_grid_chirps_df['GRIDCODE'].isin(chunk)]
    state_grid_chirps_fc_chunk = geemap.df_to_ee(state_grid_chirps_df_chunk, latitude='CHIRPS_LAT', longitude='CHIRPS_LON')
    print(state_grid_chirps_fc_chunk.size().getInfo())

    # Apply the function to each feature in the FeatureCollection
    fc_with_values = state_grid_chirps_fc_chunk.map(add_image_value)

    fc_values_df = geemap.ee_to_df(fc_with_values)

    year_interval_dfs.append(fc_values_df)

  year_interval_df = combine_dataframes(year_interval_dfs)
  ## filter out values
  year_interval_df = year_interval_df.dropna(subset=['CHIRPS_precip'])

  year_interval_df.to_csv(year_interval_csv_name, index = False)
  return(True)


Loop over years and 2-month intervals. Extract CHIRPS values. Remove CHIRPS points with NaN values. Save to csv.

In [ ]:


tracker_df = pd.read_csv(chirps_tracker, index_col=False)

for index, row in tracker_df.iterrows():
  if not row['completed']:
    current_year = row['year']
    current_interval = row['interval']

    startDate = str(current_year) + "-" + dateMap[current_interval]['startDate'];
    endDate = str(current_year) + "-" + dateMap[current_interval]['endDate'];
    layerCaption = dateMap[current_interval]['caption'];

    # print(startDate)
    # print(endDate)
    print(layerCaption + ' ' + str(current_year))

    chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate(startDate, endDate).filter(ee.Filter.calendarRange(int(startDate.split('-')[1]),int(endDate.split('-')[1]),'month'));
    chirps_sum = ee.Image(chirps.sum())


    extract_chirps_year_interval(current_year, current_interval, grid_chunks)
    ## update tracker
    tracker_df.at[index, 'completed'] = True
    ## save tracker
    tracker_df.to_csv(chirps_tracker, index = False)
    print(str(current_year) + ' ' + str(current_interval) + ' completed!')




